In [1]:
import json, psycopg2, re, networkx as nx
from datetime import datetime
from re import compile

In [2]:
with open("thanks_channel.json", encoding="UTF8") as f:
    msges = json.load(f)

In [3]:
conn = psycopg2.connect(host="localhost",
                 database="slack",
                 user="postgres",
                 password="password")

cur = conn.cursor()

In [4]:
sql_msg_table = "CREATE TABLE messages (id VARCHAR PRIMARY KEY, text VARCHAR, user_id VARCHAR, channel VARCHAR, \
                                        subtype VARCHAR, is_parent_thread BOOLEAN, post_time TIMESTAMP, edit_time TIMESTAMP);"
cur.execute(sql_msg_table)


# sql_reply_table = "CREATE TABLE replies (reply_id VARCHAR PRIMARY KEY, parent_msg_id VARCHAR REFERENCES messages (id), \
#                                         user_id VARCHAR, reply_time TIMESTAMP)"
# cur.execute(sql_reply_table)


sql_reaction_table = "CREATE TABLE reactions (user_id VARCHAR, msg_id VARCHAR REFERENCES messages (id), reaction VARCHAR);"
cur.execute(sql_reaction_table)


sql_mention_table = "CREATE TABLE mentions (msg_id VARCHAR REFERENCES messages (id), user_id VARCHAR);"
cur.execute(sql_mention_table)

In [5]:
sql_fill_msg = "INSERT INTO messages (id, text, user_id, channel, subtype, is_parent_thread, post_time, edit_time)\
VALUES (%(id)s, %(text)s, %(user_id)s, %(channel)s, %(subtype)s, %(is_parent_thread)s, %(post_time)s, %(edit_time)s);"

# sql_reply_table = "INSERT INTO replies (reply_id, parent_msg_id, user_id, reply_time)\
# VALUES (%(reply_id)s, %(parent_msg_id)s, %(user_id)s, %(reply_time)s);"

sql_fill_reaction_table = "INSERT INTO reactions (msg_id, user_id, reaction)\
VALUES (%(msg_id)s, %(user_id)s, %(reaction)s);"

sql_fill_mention_table = "INSERT INTO mentions (msg_id, user_id)\
VALUES (%(msg_id)s, %(user_id)s);"

user_pattern_in_mention = compile("(?<=<@).{9}(?=>)")

for msg in msges:
    msg_id = msg["ts"]
    text = msg["text"]
    user_id = msg.get("user")
    channel = "thanks"
    subtype = msg.get("subtype")
    is_parent_thread = (True if msg.get("thread_ts") and msg["thread_ts"] == msg["ts"] else None)
    post_time = datetime.utcfromtimestamp(float(msg_id)).strftime('%Y-%m-%d %H:%M:%S')
    if msg.get("edited"):
        edit_time = datetime.utcfromtimestamp(float(msg["edited"]["ts"])).strftime('%Y-%m-%d %H:%M:%S')
    else:
        edit_time = None
    cur.execute(sql_fill_msg, {"id": msg_id, "text": text, "user_id": user_id, "channel": channel, "subtype": subtype, 
                               "is_parent_thread": is_parent_thread, "post_time": post_time, "edit_time": edit_time})
#     if msg.get("replies"):
#         for reply in msg.get("replies"):
#             reply_time = datetime.utcfromtimestamp(float(reply["ts"])).strftime('%Y-%m-%d %H:%M:%S')
#             cur.execute(sql_reply_table, {"reply_id": reply["ts"], "parent_msg_id": msg_id,
#                                           "user_id": reply["user"], "reply_time": reply_time})
    reactions = msg.get("reactions")
    if reactions:
        for reaction in reactions:
            for user in reaction["users"]:
                cur.execute(sql_fill_reaction_table, {"msg_id": msg_id, "user_id": user, "reaction": reaction["name"]})
    for user in user_pattern_in_mention.findall(text):
        cur.execute(sql_fill_mention_table, {"msg_id": msg_id, "user_id": user})

In [12]:
cur.close()
conn.commit()
conn.close()